# this is the Capstone project for IBM Data Sicence Professional Certificate

In [2]:
#import library
import pandas as pd
import numpy as np

# Get neighborhood data

## Get neighborhood  & postal code table from wiki

In [3]:
#import module for scraping
from bs4 import BeautifulSoup
import lxml
import html5lib
import requests
print('done')

done


In [4]:
#get html from web
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')
#get the table and rows to parse
table = soup.table
table_rows=table.find_all('tr')
#create the dataframe to store the result from loop
df = {'postcode':[],'Borough':[],'Neighbourhood':[]}
Neigh = pd.DataFrame(df)
type(Neigh)


pandas.core.frame.DataFrame

In [5]:
#create the list to append to the dataframe
#loop to assign value to the list
#reference link: https://www.youtube.com/watch?v=sAuGH1Kto2I&list=PL0vuEoxnycCKWtKG-cOh1JcIwNNraQ8sP&index=4&t=0s
# the result from loop, the first result set is an empyty list as header and without index, need the try block to exclude that empty list, otherwise error
# reference link: https://www.youtube.com/watch?v=ng2o98k983k


l=[]


for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
   
    try:
        l=[{'postcode':row[0],'Borough':row[1],'Neighbourhood':row[2]}]
        Neigh=Neigh.append(l,sort=True)
    except Exception as e:
        pass
Neigh.shape

(180, 3)

In [7]:
# Drop borough that are not assigned
New = Neigh[Neigh.Borough != 'Not assigned\n']
#Remove all '\n', and replace '/' with ','
R = New.replace('\n','',regex=True)
dataframe = R.replace('/',',',regex=True)
dataframe.head()

,Borough,Neighbourhood,postcode
0,North York,Parkwoods,M3A
0,North York,Victoria Village,M4A
0,Downtown Toronto,"Regent Park , Harbourfront",M5A
0,North York,"Lawrence Manor , Lawrence Heights",M6A
0,Downtown Toronto,"Queen's Park , Ontario Provincial Government",M7A


In [8]:
dataframe.shape

(103, 3)

## Get neighborhood coordinates from web

In [9]:
co = pd.read_csv('http://cocl.us/Geospatial_data')
co.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## join two dataframe according to postcode 


In [17]:
result = pd.merge(left =dataframe, right=co, how='left',left_on='postcode',right_on='Postal Code')
Toronto_N = result[['Postal Code','Borough','Neighbourhood','Latitude','Longitude']]
Toronto_N.columns =['Postal Code','Borough','Neighborhood','Latitude','Longitude']
Toronto_N.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494


In [11]:
Toronto_N.shape

(103, 5)

# Get venue/ store information for each neighhood from Foursquare API and output a dataframe

## draw a map to see neighborhood in Toronto

In [76]:
import matplotlib.cm as cm
import matplotlib.colors as colors
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                       

In [129]:
# define latitude, longitude

latitude = 43.651070
longitude = -79.347015

# create map
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_N['Latitude'], Toronto_N['Longitude'], Toronto_N['Borough'], Toronto_N['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Get data for one neighorhood from API

In [12]:
# define API credential and version

CLIENT_ID = 'XBPSX1HKWQ3KM2NLC34PBUJDRFZYMGQUCYBSGMQK32GTXDSE' # your Foursquare ID
CLIENT_SECRET = 'TVP3QMMSOPQKFGF1CNQKAHKFWZQUJCCNV23LF5ROLLW3LRP4' # your Foursquare Secret
VERSION = '20200507' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XBPSX1HKWQ3KM2NLC34PBUJDRFZYMGQUCYBSGMQK32GTXDSE
CLIENT_SECRET:TVP3QMMSOPQKFGF1CNQKAHKFWZQUJCCNV23LF5ROLLW3LRP4


In [18]:
# get the first neighborhood name
Toronto_N.loc[0,'Neighborhood']

# get the neighborhood's latitude and longtitude values
neighborhood_latitude = Toronto_N.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Toronto_N.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Toronto_N.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [19]:
# create the get request url
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [21]:
# send the request and examine the result
results = requests.get(url).json()
type(result)

pandas.core.frame.DataFrame

In [22]:
# define a function to extract the category of the venue/ store

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [28]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # do not know why after importing pandas, still need to import this function additionally
print('done')

done


In [29]:
# clean the json file and structure it into a pandas dataframe


venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(30)

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,649 Variety,Convenience Store,43.754513,-79.331942
2,Variety Store,Food & Drink Shop,43.751974,-79.333114


## Get data for all neigborhood from API

In [31]:
# define a function to repeat the same process to all neighborhoods in Toronto

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
# run the above function on each neighborhood in Toronto and create a new dataframe called Toronto_store

Toronto_store = getNearbyVenues(names=Toronto_N['Neighborhood'],
                                   latitudes=Toronto_N['Latitude'],
                                   longitudes=Toronto_N['Longitude']
                                  )


Parkwoods
Victoria Village
Regent Park , Harbourfront
Lawrence Manor , Lawrence Heights
Queen's Park , Ontario Provincial Government
Islington Avenue
Malvern , Rouge
Don Mills
Parkview Hill , Woodbine Gardens
Garden District , Ryerson
Glencairn
West Deane Park , Princess Gardens , Martin Grove , Islington , Cloverdale
Rouge Hill , Port Union , Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate , Bloordale Gardens , Old Burnhamthorpe , Markland Wood
Guildwood , Morningside , West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor , Wilson Heights , Downsview North
Thorncliffe Park
Richmond , Adelaide , King
Dufferin , Dovercourt Village
Scarborough Village
Fairview , Henry Farm , Oriole
Northwood Park , York University
East Toronto
Harbourfront East , Union Station , Toronto Islands
Little Portugal , Trinity
Kennedy Park , Ionview , East Birchmount Park
Bayview Village
D

In [127]:
# check the resulting dataframe
print(Toronto_store.shape)
Toronto_store.columns =['Neighborhood','Neighborhood Latitude','Neighborhood Longitude','Store','Store Latitude','Store Longitude','Store Category']
Toronto_store.head(5)

(2148, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Store,Store Latitude,Store Longitude,Store Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,649 Variety,43.754513,-79.331942,Convenience Store
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [38]:
# check how many stores were returned for each neighborhood
Toronto_store.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Store,Store Latitude,Store Longitude,Store Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood , Long Branch",10,10,10,10,10,10
"Bathurst Manor , Wilson Heights , Downsview North",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park , Lawrence Manor East",24,24,24,24,24,24
Berczy Park,57,57,57,57,57,57
"Birch Cliff , Cliffside West",4,4,4,4,4,4
"Brockton , Parkdale Village , Exhibition Place",23,23,23,23,23,23
Business reply mail Processing Centre,16,16,16,16,16,16


In [39]:
# find out how many store categories
print('There are {} uniques categories.'.format(len(Toronto_store['Store Category'].unique())))

There are 266 uniques categories.


# Analyze each Neighborhood, and find the top 10 categories of each neighborhood

In [40]:
# convert category variables into indicator
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_store[['Store Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_store['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
Toronto_onehot.shape

(2148, 266)

In [42]:
# group row by neighborhood and take the mean of the frequency of occurence of each category
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1,"Alderwood , Long Branch",0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,"Bathurst Manor , Wilson Heights , Downsview North",0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,...,0.00000,0.00,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.0,0.000000
3,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,"Bedford Park , Lawrence Manor East",0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
5,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,...,0.00000,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
6,"Birch Cliff , Cliffside West",0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
7,"Brockton , Parkdale Village , Exhibition Place",0.043478,0.000000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
8,Business reply mail Processing Centre,0.062500,0.000000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
9,"CN Tower , King and Spadina , Railway Lands , ...",0.000000,0.000000,0.000000,0.062500,0.0625,0.0625,0.125,0.125,0.125,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


In [43]:
Toronto_grouped.shape

(95, 266)

In [44]:
# define a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [45]:
# create the new dataframe and display the top 10 venues for each neighborhood

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Clothing Store,Lounge,Breakfast Spot,Skating Rink,Drugstore,Eastern European Restaurant,Donut Shop,Deli / Bodega,Doner Restaurant
1,"Alderwood , Long Branch",Pizza Place,Coffee Shop,Pub,Gym,Skating Rink,Athletics & Sports,Pool,Sandwich Place,Pharmacy,Dessert Shop
2,"Bathurst Manor , Wilson Heights , Downsview North",Coffee Shop,Bank,Shopping Mall,Bridal Shop,Sushi Restaurant,Middle Eastern Restaurant,Deli / Bodega,Restaurant,Fried Chicken Joint,Pizza Place
3,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
4,"Bedford Park , Lawrence Manor East",Sandwich Place,Coffee Shop,Italian Restaurant,Restaurant,Grocery Store,Pharmacy,Juice Bar,Liquor Store,Indian Restaurant,Ice Cream Shop


# Cluster the Neighborhood

In [46]:
# import clustering library
from sklearn.cluster import KMeans

In [48]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 0, 1, 3, 1, 1, 1,
       0, 1, 3, 1, 1, 4, 3, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 3, 1, 1, 1, 3,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1,
       3, 1, 1, 1, 1, 1, 3], dtype=int32)

In [49]:
Toronto_N.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494


In [50]:
#create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_N

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3.0,Park,Food & Drink Shop,Convenience Store,Diner,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Discount Store,Curling Ice
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,French Restaurant,Coffee Shop,Intersection,Pizza Place,Hockey Arena,Portuguese Restaurant,Dessert Shop,Dance Studio,Deli / Bodega,Department Store
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,Park,Pub,Bakery,Restaurant,Breakfast Spot,Café,Theater,Hotel,Spa
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Furniture / Home Store,Women's Store,Coffee Shop,Miscellaneous Shop,Boutique,Event Space,Accessories Store,Vietnamese Restaurant,General Travel
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494,1.0,Coffee Shop,Sushi Restaurant,Creperie,Beer Bar,Sandwich Place,Burger Joint,Burrito Place,Café,Park,College Auditorium


In [128]:
# check the shape and label

Toronto_merged.shape

(100, 16)

In [124]:
# cluster label is NaN, 
Toronto_merged = Toronto_merged[Toronto_merged['Cluster Labels']!='NaN']

# convert label type into int
Toronto_merged.reset_index
Toronto_merged['Cluster Labels'] = Toronto_merged['Cluster Labels'].astype(int)

In [125]:
Toronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3,Park,Food & Drink Shop,Convenience Store,Diner,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Discount Store,Curling Ice
1,M4A,North York,Victoria Village,43.725882,-79.315572,1,French Restaurant,Coffee Shop,Intersection,Pizza Place,Hockey Arena,Portuguese Restaurant,Dessert Shop,Dance Studio,Deli / Bodega,Department Store
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,1,Coffee Shop,Park,Pub,Bakery,Restaurant,Breakfast Spot,Café,Theater,Hotel,Spa
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763,1,Clothing Store,Furniture / Home Store,Women's Store,Coffee Shop,Miscellaneous Shop,Boutique,Event Space,Accessories Store,Vietnamese Restaurant,General Travel
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Creperie,Beer Bar,Sandwich Place,Burger Joint,Burrito Place,Café,Park,College Auditorium


### Visualize clusters in map

In [126]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters


In [70]:
# cluster 1
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,North York,0.0,Pizza Place,Women's Store,Discount Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center
77,Etobicoke,0.0,Mobile Phone Shop,Bus Line,Pizza Place,Sandwich Place,Diner,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Discount Store


In [71]:
# cluster 2
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,1.0,French Restaurant,Coffee Shop,Intersection,Pizza Place,Hockey Arena,Portuguese Restaurant,Dessert Shop,Dance Studio,Deli / Bodega,Department Store
2,Downtown Toronto,1.0,Coffee Shop,Park,Pub,Bakery,Restaurant,Breakfast Spot,Café,Theater,Hotel,Spa
3,North York,1.0,Clothing Store,Furniture / Home Store,Women's Store,Coffee Shop,Miscellaneous Shop,Boutique,Event Space,Accessories Store,Vietnamese Restaurant,General Travel
4,Downtown Toronto,1.0,Coffee Shop,Sushi Restaurant,Creperie,Beer Bar,Sandwich Place,Burger Joint,Burrito Place,Café,Park,College Auditorium
7,North York,1.0,Beer Store,Sporting Goods Shop,Restaurant,Gym,Coffee Shop,Asian Restaurant,Japanese Restaurant,Chinese Restaurant,Supermarket,Café
8,East York,1.0,Pizza Place,Fast Food Restaurant,Bank,Gym / Fitness Center,Athletics & Sports,Pet Store,Gastropub,Pharmacy,Intersection,Dim Sum Restaurant
9,Downtown Toronto,1.0,Clothing Store,Coffee Shop,Café,Japanese Restaurant,Restaurant,Cosmetics Shop,Italian Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Tea Room
10,North York,1.0,Pub,Bakery,Pizza Place,Japanese Restaurant,Women's Store,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
12,Scarborough,1.0,Bar,Golf Course,Moving Target,Women's Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
13,North York,1.0,Beer Store,Sporting Goods Shop,Restaurant,Gym,Coffee Shop,Asian Restaurant,Japanese Restaurant,Chinese Restaurant,Supermarket,Café


In [72]:
# cluster 3
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Etobicoke,2.0,Filipino Restaurant,Women's Store,Curling Ice,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


In [75]:
# cluster 4
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]].shape

(12, 12)

In [74]:
# cluster 5
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,4.0,Fast Food Restaurant,Women's Store,Curling Ice,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
